In [1]:
import cv2
#https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
import dlib
import numpy as np

In [21]:
cap = cv2.VideoCapture(0)

In [22]:
#一个简单的cv2读取视频流
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read() #读取，ret代表着读取frame的一些参数

    # Our operations on the frame come here

    # Display the resulting frame
    cv2.imshow('frame', frame) # 展示图像
    if cv2.waitKey(1) & 0xFF == ord('q'): #按q退出
        break
cap.release() #清楚
cv2.destroyAllWindows() #关掉window

In [2]:
# 结合上次我们的dlib
detector = dlib.get_frontal_face_detector() #跟opencv的haar作用一样，用来检测人脸
predictor = dlib.shape_predictor("first/shape_predictor_68_face_landmarks.dat") 
def cal_eye_dist(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) < 1:
        return 0
    landmarks = predictor(image=gray, box=faces[0]) #传入模型得到结果
    left_one = cal_e_dist(landmarks.part(37).x, landmarks.part(41).x,
                         landmarks.part(37).y, landmarks.part(41).y)/img.shape[0]
    left_two = cal_e_dist(landmarks.part(38).x, landmarks.part(40).x,
                         landmarks.part(38).y, landmarks.part(40).y)/img.shape[0]
    right_one = cal_e_dist(landmarks.part(43).x, landmarks.part(47).x,
                         landmarks.part(43).y, landmarks.part(47).y)/img.shape[0]
    right_two = cal_e_dist(landmarks.part(44).x, landmarks.part(46).x,
                         landmarks.part(44).y, landmarks.part(46).y)/img.shape[0]
    return round(left_one,4), round(left_two,4), round(right_one,4), round(right_two,4)

In [3]:
def cal_e_dist(x1, x2, y1, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

## 运行下面代码的时候应该会出现一个小窗口实时显示摄像头数据，摄像头上的数字是函数返回的四个值（每一个frame)

In [33]:
# 结合dlib
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read() #读取，ret代表着读取frame的一些参数

    # Our operations on the frame come here
    dst = cal_eye_dist(frame)
    frame = cv2.putText(frame,str(dst),(10,200), font, 1,(255,255,255),1)
    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): #按q退出
        break
cap.release() #清楚
cv2.destroyAllWindows() #关掉window

In [4]:
import time

In [16]:
# 现在我们尝试复杂点
# 前面3s帧用来设立基准线
# fps 手动设置为10
frame_rate = 10
prev = 0
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
count = 0
baseline = [] #创建数组储存返回数据
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read() #读取，ret代表着读取frame的一些参数
    time_elapsed = time.time() - prev #时间差
    #print(time_elapsed)
    if count <= 30: #3s前用来设置baseline
        if time_elapsed > 1./frame_rate: #时间差大于我们设置好的1/fps时执行操作
            count += 1
            dst = cal_eye_dist(frame)
            #print(np.mean(dst))
            baseline.append(np.mean(dst))
            frame_now = cv2.putText(frame,'Do not blink! Setting the baseline',(10,200), font, 1,(255,255,255),1)
            #print(frame_b)
            # Display the resulting frame
            cv2.imshow('frame', frame_now)
            cv2.waitKey(1) 
            prev = time.time()
        continue
    thre = np.mean(baseline) * 0.6        # using 0.6 mean as threshold. 
    if time_elapsed > 1./frame_rate: #时间差大于我们设置好的1/fps时执行操作
        dst = cal_eye_dist(frame)
        #print(thre, dst)
        if np.mean(dst) < thre:
            frame_now = cv2.putText(frame,'Warning! Tired',(10,200), font, 1,(255,255,255),1)
        else:
            frame_now = cv2.putText(frame,'Nothing Happened',(10,200), font, 1,(255,255,255),1)
        # Display the resulting frame
        cv2.imshow('frame', frame_now)
        prev = time.time()
    if cv2.waitKey(1) & 0xFF == ord('q'): #按q退出
        break
cap.release() #清楚
cv2.destroyAllWindows() #关掉window